
Mounting google drive

In [ ]:
import os
import pandas as pd
import cv2
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


'/content/drive/MyDrive/DeepFake dta/diffusion_datasets'

In [ ]:
!pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-qhi__3e_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-qhi__3e_
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 2.0 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=74498233b8fc0e10e6bef68bfdf3c37263dc66e65242cac37d911c540b613fd2
  Stored in directory: /tmp/pip-ephem-wheel-cache-mr4683vf/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


Load CLIP-ViT-L/14 model to encode images, preprocessing images with gaussisan blur and JPEG compression and encode them

In [ ]:
import os
import torch
import numpy as np
from PIL import Image, ImageFilter
import io
import clip
import cv2
from google.colab import drive

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, transform = clip.load("ViT-L/14", device=device)

# Function for Gaussian blur and JPEG compression
def preprocess(image, blur_radius=5, jpeg_quality=85):
    blurred_image = cv2.GaussianBlur(image, (blur_radius, blur_radius), 0)
    pil_image = Image.fromarray(cv2.cvtColor(blurred_image, cv2.COLOR_BGR2RGB))
    image_bytes = io.BytesIO()
    pil_image.save(image_bytes, 'JPEG', quality=jpeg_quality)
    return image_bytes.getvalue()

# Function to encode images one by one
def load_encode_recursive(folder_path, model, transform):
    features_list, labels_list, fake_list = [], [], []

    for class_label, class_name in enumerate(['real', 'fake']):
        class_folder = os.path.join(folder_path, class_name)

        for root, dirs, files in os.walk(class_folder):
            for filename in files:
                image_path = os.path.join(root, filename)

                image = cv2.imread(image_path)
                preprocessed_image = preprocess(image)

                preprocessed_pil_image = Image.open(io.BytesIO(preprocessed_image))
                transformed_image = transform(preprocessed_pil_image).unsqueeze(0).to(device)

                features = model.encode_image(transformed_image)
                features_np = features.detach().cpu().numpy()

                features_list.append(features_np)
                labels_list.append([class_label, root])

    return np.concatenate(features_list), np.array(labels_list), fake_list


# Load, preprocess, and encode images
all_features, all_labels, fake_features = load_encode_recursive('/content/drive/MyDrive/Deepfake', model, transform)


Mounted at /content/drive


100%|███████████████████████████████████████| 890M/890M [00:10<00:00, 92.8MiB/s]


In [ ]:
labels = [int(all_labels[i][0]) for i in range(len(all_labels))]
classes = [all_labels[i][1] for i in range(len(all_labels))]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Adding encodings of deepfake images of different generative models for training SVMs, Random Forest and Neural Network for chain classification

In [ ]:
uniq_class = set(classes)
uniq_class1 = []
ll = ['/content/drive/MyDrive/Deepfake/fake/fake1', '/content/drive/MyDrive/Deepfake/fake/fake6', '/content/drive/MyDrive/Deepfake/fake/fake2/1_fake', '/content/drive/MyDrive/Deepfake/fake/fake7', '/content/drive/MyDrive/Deepfake/fake/fake3', '/content/drive/MyDrive/Deepfake/fake/fake2', '/content/drive/MyDrive/Deepfake/fake/fake4', '/content/drive/MyDrive/Deepfake/fake/fake5']
for clss in uniq_class:
  if clss in ll:
    uniq_class1.append(clss)
feat_dict = {}
for clss in uniq_class1:
  feat_dict[clss] = []
for ind, feature in enumerate(all_features):
  if all_labels[ind][1] in uniq_class1:
    feat_dict[all_labels[ind][1]].append(feature)


Adding a layer of Neural Network to classify image as real or fake

In [ ]:
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(all_features, labels, test_size=0.2, random_state=42)

# Define the linear classifier model
class LinearClassifier(nn.Module):
    def __init__(self, input_size):
        super(LinearClassifier, self).__init__()
        self.fc = nn.Linear(input_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

# Initialize the linear classifier
input_size = all_features.shape[1]
classifier = LinearClassifier(input_size).to(device)

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

# Convert the data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train).to(device)
y_train_tensor = torch.FloatTensor(y_train).unsqueeze(1).to(device)

# Training loop
epochs = 1000
for epoch in range(epochs):
    # Forward pass
    outputs = classifier(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')

# Evaluate the model on the test set
X_test_tensor = torch.FloatTensor(X_test).to(device)
y_test_tensor = torch.FloatTensor(y_test).unsqueeze(1).to(device)

with torch.no_grad():
    test_outputs = classifier(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    print(f'Test Loss: {test_loss.item()}')


Epoch [1/1000], Loss: 0.8863220810890198
Epoch [2/1000], Loss: 0.8139565587043762
Epoch [3/1000], Loss: 0.7476633787155151
Epoch [4/1000], Loss: 0.6876567602157593
Epoch [5/1000], Loss: 0.6340117454528809
Epoch [6/1000], Loss: 0.5866555571556091
Epoch [7/1000], Loss: 0.5453728437423706
Epoch [8/1000], Loss: 0.5098185539245605
Epoch [9/1000], Loss: 0.47954341769218445
Epoch [10/1000], Loss: 0.45402559638023376
Epoch [11/1000], Loss: 0.4327049255371094
Epoch [12/1000], Loss: 0.4150128960609436
Epoch [13/1000], Loss: 0.40039774775505066
Epoch [14/1000], Loss: 0.3883429765701294
Epoch [15/1000], Loss: 0.37838003039360046
Epoch [16/1000], Loss: 0.3700948655605316
Epoch [17/1000], Loss: 0.36313027143478394
Epoch [18/1000], Loss: 0.35718441009521484
Epoch [19/1000], Loss: 0.35200726985931396
Epoch [20/1000], Loss: 0.34739547967910767
Epoch [21/1000], Loss: 0.3431870937347412
Epoch [22/1000], Loss: 0.33925575017929077
Epoch [23/1000], Loss: 0.33550530672073364
Epoch [24/1000], Loss: 0.33186501

In [ ]:
fake_features = feat_dict.values()
fake_features = list(fake_features)

(1000, 768)


In [ ]:
import random
from sklearn.model_selection import train_test_split

fake_data = []
y_data = []

# Storing images' encoding in a matrix
for class_data in enumerate(fake_features):
  temp_feat = np.array(class_data[1])
  temp_label = np.array([1 for i in range(len(class_data[1]))])
  for j in range(len(fake_features)):
    if j != class_data[0]:
      random_ele = np.array(random.sample(fake_features[j], len(class_data[1])//len(uniq_class)))
      random_label = np.array([0 for i in range(len(class_data[1])//len(uniq_class))])
      temp_feat = np.concatenate((temp_feat, random_ele))
      temp_label = np.concatenate((temp_label, random_label))
  fake_data.append(np.matrix(temp_feat))
  y_data.append(temp_label)


(8, 1700)


Using encoded images to create test, train and validate dataset from 4 generative models to be used for chain classifiers

In [ ]:
from sklearn.model_selection import train_test_split

fake_datasets = fake_data
y_datasets = y_data
num_datasets = len(fake_datasets)
X_train_datasets = []
X_test_datasets = []
y_train_datasets = []
y_test_datasets = []

# Diving into test, train and validate sets
seed_value = 42
for i in range(num_datasets):
    X_train, X_test, y_train, y_test = train_test_split(
        fake_datasets[i], y_datasets[i], test_size=0.2, random_state=seed_value
    )
    X_train_datasets.append(np.array(X_train))
    X_test_datasets.append(np.array(X_test))
    y_train_datasets.append(np.array(y_train))
    y_test_datasets.append(np.array(y_test))
    seed_value += 1


In the following cells SVMs, NN, Random forests were trained on these datasets from different models for chain classification, each model is trained only on datasets from 1 model

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X1_scaled = scaler.fit_transform(X_train_datasets[4])
X2_scaled = scaler.fit_transform(X_train_datasets[5])
X3_scaled = scaler.fit_transform(X_train_datasets[6])
X4_scaled = scaler.fit_transform(X_train_datasets[7])

# Create SVM instances for each dataset
svm1 = svm.SVC(kernel='sigmoid')
svm2 = svm.SVC(kernel='sigmoid')
svm3 = svm.SVC(kernel='sigmoid')
svm4 = svm.SVC(kernel='sigmoid')

# Train SVM models on each dataset
svm1.fit(X1_scaled, y_train_datasets[4])
svm2.fit(X2_scaled, y_train_datasets[5])
svm3.fit(X3_scaled, y_train_datasets[6])
svm4.fit(X4_scaled, y_train_datasets[7])

y1_pred = svm1.predict(scaler.transform(X_test_datasets[4]))
y2_pred = svm2.predict(scaler.transform(X_test_datasets[5]))
y3_pred = svm3.predict(scaler.transform(X_test_datasets[6]))
y4_pred = svm4.predict(scaler.transform(X_test_datasets[7]))

# Evaluate accuracy
acc1 = accuracy_score(y_test_datasets[4], y1_pred)
acc2 = accuracy_score(y_test_datasets[5], y2_pred)
acc3 = accuracy_score(y_test_datasets[6], y3_pred)
acc4 = accuracy_score(y_test_datasets[7], y4_pred)

# Display results
print(f"Accuracy for Dataset 1: {acc1}")
print(f"Accuracy for Dataset 2: {acc2}")
print(f"Accuracy for Dataset 3: {acc3}")
print(f"Accuracy for Dataset 4: {acc4}")


Accuracy for Dataset 1: 0.7588235294117647
Accuracy for Dataset 2: 0.7970588235294118
Accuracy for Dataset 3: 0.7147058823529412
Accuracy for Dataset 4: 0.8235294117647058


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Train four random forests with different settings
rf1 = RandomForestClassifier(n_estimators=50, random_state=1)
rf2 = RandomForestClassifier(n_estimators=100, random_state=2)
rf3 = RandomForestClassifier(n_estimators=150, random_state=3)
rf4 = RandomForestClassifier(n_estimators=200, random_state=4)

# Train the models
rf1.fit(X_train_datasets[4], y_train_datasets[4])
rf2.fit(X_train_datasets[5], y_train_datasets[5])
rf3.fit(X_train_datasets[6], y_train_datasets[6])
rf4.fit(X_train_datasets[7], y_train_datasets[7])

# Make predictions on the test set
y_pred1 = rf1.predict(X_test_datasets[4])
y_pred2 = rf2.predict(X_test_datasets[5])
y_pred3 = rf3.predict(X_test_datasets[6])
y_pred4 = rf4.predict(X_test_datasets[7])

# Evaluate the models
acc1 = accuracy_score(y_test_datasets[4], y_pred1)
acc2 = accuracy_score(y_test_datasets[4], y_pred2)
acc3 = accuracy_score(y_test_datasets[4], y_pred3)
acc4 = accuracy_score(y_test_datasets[4], y_pred4)

print("Accuracy for Random Forest 1:", acc1)
print("Accuracy for Random Forest 2:", acc2)
print("Accuracy for Random Forest 3:", acc3)
print("Accuracy for Random Forest 4:", acc4)


Accuracy for Random Forest 1: 0.7235294117647059
Accuracy for Random Forest 2: 0.55
Accuracy for Random Forest 3: 0.55
Accuracy for Random Forest 4: 0.5882352941176471


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def create_neural_network():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_datasets[4].shape[1],)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')  # Assuming you have 3 classes for classification
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

nn1 = create_neural_network()
nn2 = create_neural_network()
nn3 = create_neural_network()
nn4 = create_neural_network()

# Training
nn1.fit(X_train_datasets[4], y_train_datasets[4], epochs=10, batch_size=32, validation_split=0.2)
nn2.fit(X_train_datasets[5], y_train_datasets[5], epochs=10, batch_size=32, validation_split=0.2)
nn3.fit(X_train_datasets[6], y_train_datasets[6], epochs=10, batch_size=32, validation_split=0.2)
nn4.fit(X_train_datasets[7], y_train_datasets[7], epochs=10, batch_size=32, validation_split=0.2)

# Make predictions on the test set
y_pred_prob_nn1 = nn1.predict(X_test_datasets[4])
y_pred_prob_nn2 = nn2.predict(X_test_datasets[5])
y_pred_prob_nn3 = nn3.predict(X_test_datasets[6])
y_pred_prob_nn4 = nn4.predict(X_test_datasets[7])

# Get the predicted classes
y_pred_nn1 = tf.argmax(y_pred_prob_nn1, axis=1)
y_pred_nn2 = tf.argmax(y_pred_prob_nn2, axis=1)
y_pred_nn3 = tf.argmax(y_pred_prob_nn3, axis=1)
y_pred_nn4 = tf.argmax(y_pred_prob_nn4, axis=1)

# Evaluate the models
acc_nn1 = accuracy_score(y_test_datasets[4], y_pred_nn1)
acc_nn2 = accuracy_score(y_test_datasets[4], y_pred_nn2)
acc_nn3 = accuracy_score(y_test_datasets[4], y_pred_nn3)
acc_nn4 = accuracy_score(y_test_datasets[4], y_pred_nn4)

print("Accuracy for Neural Network 1:", acc_nn1)
print("Accuracy for Neural Network 2:", acc_nn2)
print("Accuracy for Neural Network 3:", acc_nn3)
print("Accuracy for Neural Network 4:", acc_nn4)


Epoch 1/10
34/34 [==============================] - 2s 16ms/step - loss: 0.6584 - accuracy: 0.6608 - val_loss: 0.5496 - val_accuracy: 0.7390
Epoch 2/10
34/34 [==============================] - 0s 9ms/step - loss: 0.4666 - accuracy: 0.8006 - val_loss: 0.5304 - val_accuracy: 0.7537
Epoch 3/10
34/34 [==============================] - 0s 11ms/step - loss: 0.3891 - accuracy: 0.8346 - val_loss: 0.5189 - val_accuracy: 0.7500
Epoch 4/10
34/34 [==============================] - 0s 8ms/step - loss: 0.3195 - accuracy: 0.8778 - val_loss: 0.5369 - val_accuracy: 0.7684
Epoch 5/10
34/34 [==============================] - 0s 8ms/step - loss: 0.2570 - accuracy: 0.9062 - val_loss: 0.5524 - val_accuracy: 0.7647
Epoch 6/10
34/34 [==============================] - 0s 14ms/step - loss: 0.1857 - accuracy: 0.9357 - val_loss: 0.6775 - val_accuracy: 0.7132
Epoch 7/10
34/34 [==============================] - 0s 7ms/step - loss: 0.1412 - accuracy: 0.9577 - val_loss: 0.6604 - val_accuracy: 0.7426
Epoch 8/10
34/34 